(dUcluster)=
# Datos de UCluster
El pre-procesamiento de los datos de cada participante de las LHCO 2020 es distinto. Algunos métodos requieren la creación de imágenes, otros el cálculo de variables, y el uso de estas variables también depende del algoritmo a utilizar. En esta sección vamos a analizar el pre-procesamiento de los datos hecho para UCluster utilizando el conjunto R&D.

(dUcluster-dist)=
## Distribuciones
Las variables de los datos pre-procesados por UCluster se describen en la {numref}`datoslhco-ucluster-variables`{cite}`Mikuni_2021`:

```{table} Definición de las variables utilizadas por UCluster para entrenamiento y clasificación.
:name: datoslhco-ucluster-variables
|Variable                    |Descripción                                                                   |
|:--------------------------:|:----------------------------------------------------------------------------:|
|$\Delta\eta$                | Diferencia entre la pseudo-rapidez del constituyente y del jet               |
|$\Delta\phi$                | Diferencia entre el ángulo azimutal del constituyente y del jet              |
|$\log{p_T}$                 | Logaritmo del $p_T$ del constituyente                                        |
|$\log{E}$                   | Logaritmo de la $E$ del constituyente                                        |
|$\log\frac{p_T}{p_{T_{jet}}}$| Logaritmo de la relación entre el $p_T$ del constituyente y el $p_T$ del jet |
|$\log\frac{E}{E_{jet}}$     | Logaritmo de la relación entre la $E$ del constituyente y la $E$ del jet     |
|$\Delta R$                  | Distancia entre el constituyente y el jet en el espacio $\eta-\phi$          |
|PID                         | Identificador del tipo de partícula                                          |
```
La distribución de estas variables se puede observar en {numref}`dUCluster-dist-data`. La distribución de $\eta$ es igual para señal y fondo. $\phi$ y $\Delta R$ poseen una distribución de señal más angosta y con una mayor densidad de eventos que el fondo, por lo explicado en la {numref}`datospp-dist`. Las distribuciones relacionadas a $p_T$ y $E$ son similares, y se observan diferencias entre señal y fondo. En ambas variables, los picos de los eventos de señal se encuentran en valores mayores de $p_T$ que los de fondo, debido a la mayor energía de la interacción, como se explicó anteriormente.

In [2]:
# Importamos librerias básicas
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import nexusformat.nexus as nx

# Funciones de benchtools
from benchtools.src.plotools import bkg_sig_hist, bkg_sig_scatter, create_png, image_grid

# Definimos variables globales
PATH_IMAGES='../../figuras/'

In [4]:
# Cargamos el archivo de los datos de UCluster
# Hay varios dataframes en el archivo, aqui se carga "data"
with h5py.File('../../../datos/train_20v_RD_100P_2NJET.h5', 'r') as hdf:
    hdfdata = hdf.get('data')
    data = np.array(hdfdata)

In [5]:
# Convertimos los datos en dataframe
columns = ['eta', 'phi', 'ratio_pT', 'ratio_E', 'pT', 'E', 'delta_R']
# Calculamos la media de cada variable, por evento
df = pd.DataFrame(data.mean(axis=(1)), columns = columns)
# Agregamos la etiqueta de señal o fondo
pid = pd.Series(np.array(h5py.File('../../../datos/train_20v_RD_100P_2NJET.h5', 'r')['pid']))
df = df.assign(label=pid.values)

# Graficamos
variables = df.drop('label', axis=1).columns
lista = create_png(namedf='UCluster', df=df, variables=variables, path=PATH_IMAGES, keyname='dist', nbins=50, type='distribution',title=True)
# Grid de imagenes
image_grid(rows=2, columns=4, images=lista, name='dUCluster-dist-data', path=PATH_IMAGES, remove=True)


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/dUCluster-dist-data.png
---
name: dUCluster-dist-data
width: 100%
---
Distribución de las variables en la {numref}`datoslhco-ucluster-variables`
```
Para UCluster también se calculan variables globales del evento, que son utilizadas para parametrizar la red. Estas son: el logaritmo de la masa de los dos jets principales y $\tau_{21}$ de los dos jets principales ({numref}`dUCluster-dist-global`). La distribución de $\tau_{21}$ es similar a la obtenida con `benchtools`, analizada en la {numref}`datospp-dist`.

In [6]:
# Obtenemos el dataframe "global"
columns=['mass_j1', 'tau12_j1', 'mass_j2', 'tau12_j2']
df_global = pd.DataFrame(np.array(h5py.File('../../../datos/train_20v_RD_100P_2NJET.h5', 'r')['global']), columns=columns)
df_global=df_global.assign(label=pid.values)

# Graficamos
lista = create_png(namedf='UCluster', df=df_global, variables=columns, path=PATH_IMAGES, keyname='global', nbins=50, type='distribution', title=True)
# Grid de imagenes
image_grid(rows=2, columns=2, images=lista, name='dUCluster-dist-global', path=PATH_IMAGES, remove=True)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/dUCluster-dist-global.png
---
name: dUCluster-dist-global
width: 70%
---
Distribución de las variables para parametrizar la red de UCluster.
```

El conjunto de datos pre-procesados también posee un conjunto *label*, que hace referencia a las etiquetas de masa asignadas a los constituyentes de los jets. De la reconstrucción de esta variable, considerando el rango de masa utilizado por el método{cite}`Kasieczka_2021`, se obtiene la {numref}`dUCluster-label`, donde se observan dos picos alrededor de la masa de las partículas $X$ y $Y$ del conjunto R&D.

In [7]:
# Cargamos el dataframe de label
df_label = pd.DataFrame(np.array(h5py.File('../../../datos/train_20v_RD_100P_2NJET.h5', 'r')['label']))
# Agregamos las etiquetas de señal y fondo
df_label = df_label.assign(label=pid.values)
# Separamos señal y fondo
df_label_bkg = df_label.loc[df_label['label']==0]
df_label_sig = df_label.loc[df_label['label']==1]

# Hacemos un mapeo entre la etiqueta y la masa
# Creamos el rango de masas y de etiquetas
MASSRANGE = np.linspace(10,1000,20)
MASSRANGE = np.append(MASSRANGE, [100000])
map_list = []
for x in enumerate(MASSRANGE):
    if x[0]==20: break
    map_list.append(x)
df_map = pd.DataFrame(map_list, columns=['value', 'mass'])
df_map.head()

# Mapeamos en el dataframe las etiquetas con los valores de masa
df_map = df_map.set_index("value")["mass"]
df_label_map = pd.DataFrame()
for col in df_label.drop('label', axis=1):
    df_label_map[col] = df_label[col].map(df_map).fillna(df_label[col])
# Agregamos las etiquetas de señal y fondo a este dataframe
df_label_map = df_label_map.assign(label=pid.values)
df_label_map.head()
# Separamos
df_label_bkg = df_label_map.loc[df_label_map['label']==0]
df_label_sig = df_label_map.loc[df_label_map['label']==1]
# Graficamos
fig = plt.figure(facecolor='white')
# 'stack' para graficar todos los valores de dataframe en el histograma 
df_label_bkg.drop('label', axis=1).stack().plot.hist(facecolor='b', alpha=0.2, label='background', density=True)
df_label_sig.drop('label', axis=1).stack().plot.hist(facecolor='r', alpha=0.2, label='signal', density=True)
plt.legend(loc='upper right')
plt.title('Distribución de la variable "label"');
# Guardamos la figura
plt.savefig('../../figuras/dUCluster-label.png', bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
plt.close()

```{figure} ./../../figuras/dUCluster-label.png
---
name: dUCluster-label
width: 50%
---
Distribución de la media de las etiquetas de masa asignadas a los constituyentes de los jets. Los picos de la señal coinciden con la masa de las partículas $X$ y $Y$ para el conjunto analizado.
```
(dUCluster-corr)=
## Correlaciones
En la {numref}`dUCluster-correlaciones`, se presenta el mapa de correlaciones de las variables utilizadas por UCluster.

In [8]:
# Juntamos los dataframes con las variables
df_all = pd.concat([df, df_global], axis=1)
# Eliminamos la columna 'label' duplicada
df_all = df_all.loc[:,~df_all.columns.duplicated()]

# Queremos tener las correlaciones para la señal y el fondo en dataframes separados
df_sig = df_all[df_all['label']==1].drop(['label'], axis=1)
df_bkg = df_all[df_all['label']==0].drop(['label'], axis=1)

# Calculamos las correlaciones
df_sig_corr = df_sig.corr()
df_bkg_corr = df_bkg.corr()

# Definimos figura y tamaño
fig, axes = plt.subplots(figsize=[14,11], facecolor='white', nrows=1, ncols=2, sharey=True)
fig.tight_layout() 

# Para graficar el mapa de correlaciones:
for ax, (name, corr) in zip(axes.flat, [('Señal', df_sig_corr), ('Fondo', df_bkg_corr)]):
    
    cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)

    #Ejes
    ticks = np.arange(0,len(corr.columns),1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(corr.columns, rotation=90)
    ax.set_yticklabels(corr.columns)
    ax.xaxis.tick_top()
    ax.set_title('{}'.format(name), fontsize=16)
    
    
# Mapa de color
fig.colorbar(cax, ax=axes.ravel().tolist(), fraction=0.025, pad=0.04)
# Título
#fig.suptitle('Mapa de correlaciones', fontsize=25)

#plt.show()
plt.savefig('./../../figuras/dUCluster-correlaciones', bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
plt.close()


```{figure} ./../../figuras/dUCluster-correlaciones.png
---
name: dUCluster-correlaciones
width: 80%
---
Correlaciones de las variables obtenidas del pre-procesamiento realizado para UCluster.
```
Se observa que en los eventos de señal y fondo existen correlaciones fuertes entre $ratio\_E$ y $ratio\_pT$ y entre $E$ y $pT$. Esta relación es natural debido a que $E$ se calcula con las componentes del momento. En el fondo, se observan correlaciones entre la masa del jet principal y secundario con $pT$. Sin embargo, no son evidentes en la {numref}`dUCluster-bkgcorr`.

In [9]:
variables = [('mass_j1', 'ratio_pT'), ('mass_j2', 'ratio_pT')]
lista = create_png(namedf='UCluster', df=df_all, variables=variables, path=PATH_IMAGES, keyname='bkgcorr', type='scatter', title=True)
image_grid(rows=1, columns=2, images=lista, name='dUCluster-bkgcorr', path=PATH_IMAGES, remove=True)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/dUCluster-bkgcorr.png
---
name: dUCluster-bkgcorr
width: 100%
---
Correlaciones de las masa de los jets principales con $ratio\_pT$.
```